# Assignment 02
Welcome to our cs334 assignment 02. In this assignment you are going to implement the basic forward kinematic and inverse kinematic algorithms we discussed in the previous lecture.

Please complete this assignment by following the instruction in each block step by step. There are 3 blocks that your need to fill in, and you MUST implement your solution within the range of Implementation Block. The location of the Implementation Block is labeled by comments: "Implementation Block of \<algorithm\> starts here" and "Implementation Block of \<algorithm\> ends here"
- You CAN NOT create any new blocks.
- You CAN NOT modify any blocks or codes.
- You CAN NOT create any extra files.
- You CAN NOT use any third party library, except for numpy and matplotlib
- You CAN NOT write any code outside the range of Implementation Block.
 > Release: Tuesday, Mar, 5, 2024

> Due: Thursday, 3:00 pm, Mar 7, 2024 (Before Class)

## Online Environment
you can upload this notebook to google colab, so you can run everything online (you may miss demo gifs in google colab, but you can check them in local)

## Submission
1. only on .zip file should be uploaded, the following files shoulbe be included the .zip file
   1. assignment02.ipynb (this file: -100 points if missing it)
   2. task1.gif (20 points)
   3. task2_1.gif (20 points)
   4. task2_2.gif (20 points)
   5. task3_1.gif (20 points)
   6. task3_2.gif (20 points)
   7. extra_points.gif (30 points) 
2. missing any of the file will lead you to lose its corresponding points.
3. any extra files submitted will be ignored.

In [ ]:
# run the following command to install the required packages
! pip install numpy
! pip install matplotlib

In [ ]:
from typing import List
import numpy as np
from IPython.display import HTML
import utils

%load_ext autoreload
%autoreload 2
%matplotlib inline  


# task 1: forward kinematics (20 points)
please implement your fabrik algorithm in the block below (within the implementation block).
1. your implementation should support fk in 3d space, including 3d arms and 3d target.
2. your implementation should support bones with arbitrary length. 
3. Attention: position of arm and direction of axis are defined in global space
- you can use utils.normalize to normalize a vector
- you can use utils.length to compute the length of a vector
- you can use utils.rotation_mat_3d to get the rotation matrix of a 3d rotation 
- you can use utils.angle_between3d to get the angle between 2 vectors.




In [ ]:
def forward_kinematic_3d(arm:np.ndarray,axis:np.ndarray, angles:List[float]):
    """
    Implement the forward kinematic for a 3D arm
    :param arm: np.ndarray of shape (n,3) representing the arm with n nodes in global position
    :param axis: np.ndarray of shape (n-1,3) representing the axis of rotation for each bone in GLOBAL SPACE
    :param angles: np.ndarray of shape (n-1,) representing the angle of rotation for each bone
    :return: np.ndarray of shape (n,3) representing the position of each bone
    """
    num_nodes = arm.shape[0]
    assert arm.shape == (num_nodes, 3), f"Expected arm to be of shape {(num_nodes, 3)} but got {arm.shape}"
    assert axis.shape == (num_nodes - 1, 3), f"Expected axis to be of shape {(num_nodes - 1, 3)} but got {axis.shape}"
    assert len(angles)== num_nodes-1, f"Expected angles to be of length {num_nodes-1} but got {len(angles)}"
    ########## Implementation Block of FK algorithm starts here ##########
    # your code here, official solution has 8 lines of code (not optimized for line count)
  
    ########## Implementation Block of FK algorithm ends here ##########
    return arm

## task 1 test case 0: 
- Run the test case below to test your solution
- You should be able to see the following gif after running the block successfully. (maybe different play speed)

 
![fk3d.gif](fk3d.gif)

In [ ]:
arm = np.array([[0.0, 0.0, 0.0], [0.0, 0.5, 0.0], [0.0, 1.0, 0.0],[0.0, 2.0, 0.0]])
init_axis = np.array([[1.0,0.0, 0.0], [0.0, 1.0, 0.0],[0.0,0.0,1.0]])
delta_angles = [-0.1, 0.1,0.05]
timesteps = 200
ik_arms = []

for i in range(timesteps):
    arm = forward_kinematic_3d(arm,init_axis,delta_angles)
    ik_arms.append(arm)
ik_arms = np.array(ik_arms)
print("generating animations")
ani = utils.plot_nodes_trajs(ik_arms,"fk3d.gif")
ik_end_targets = ik_arms[:, -1]
HTML(data=ani.to_jshtml())

# task 1 result generation
Run the following block which generates a gif "task1.gif".

In [ ]:
arm = np.array([[0.0, 0.0, 0.0], [0.0, 0.5, 0.0], [0.0, 1.0, 0.0], [0.0, 2.0, 0.0]])
axis = np.array([[0.0, 0.0, 1.0], [1.0, 1.0, 1.0], [0.0, 0.0, 1.0]])
delta_angles = [0.1, -0.05, -0.1]
timesteps = 200
ik_arms = []

for i in range(timesteps):
    arm = forward_kinematic_3d(arm, axis, delta_angles)
    ik_arms.append(arm)
ik_arms = np.array(ik_arms)
print("generating animations task1.gif")
ani = utils.plot_nodes_trajs(ik_arms,"task1.gif")
# ik_end_targets = ik_arms[:, -1]
HTML(data=ani.to_jshtml())

# task 2: fabrik algorithm (40 points)
please implement your fabrik algorithm in the block below (within the implementation block).
1. your implementation should support ik in 3d space, including 3d arms and 3d target. (20 points)
2. your implementation should support bones with arbitrary length. (20 points)

- you can use utils.normalize to normalize a vector
- you can use utils.length to compute the length of a vector
- you can use utils.rotation_mat_3d to get the rotation matrix of a 3d rotation 
- you can use utils.angle_between3d to get the angle between 2 vectors.


> reference: http://www.andreasaristidou.com/publications/papers/FABRIK.pdf

In [ ]:
def fabrik_3d(arm:np.ndarray, end_target:np.ndarray, max_iter:int = 1000,precision:float = 0.01) -> np.ndarray:
    """
    Implement the FABRIK algorithm for a 3D arm
    :param arm: np.ndarray of shape (n,3) representing the joints of the arm (the first joint is at the origin and static)
    :param end_target: np.ndarray of shape (3,) representing the end effector target
    :param max_iter: int, maximum number of iterations
    :param precision: float, tolerance for the distance between the end effector and the target
    :return: np.ndarray of shape (n,3) representing the new joints of the arm
    """
    num_joints = arm.shape[0]
    assert arm.shape == (num_joints, 3), f"Expected arm to be of shape {(num_joints, 3)} but got {arm.shape}"
    assert end_target.shape == (3,), f"Expected end_target to be of shape (3,) but got {end_target.shape}"

    ########## Implementation Block of FABRIK algorithm starts here ##########
    # your code here, solution is 16 lines of code (not optimized for line count)
    
    ########## Implementation Block of FABRIK algorithm ends here ##########
    arm = np.array(arm)
    assert arm.shape == (num_joints, 3), f"Expected IK arm to be of shape {(num_joints, 3)} but got {arm.shape}"
    return arm

## task 2 test case 0: 
- Run the test case below to test your solution
- You should be able to see the following gif after running the block successfully. (maybe different play speed)
 
![fabrik3d_fktargets.gif](fabrik3d_fktargets.gif)

In [ ]:
# arm = np.array([[0.0, 0.0, 0.0], [0.0, 0.5, 0.0], [0.0, 1.5, 0.0], [0.0, 2.0, 0.0]])
arm = np.array([[0.0, 0.0, 0.0], [0.0, 0.5, 0.0], [0.0, 1.0, 0.0], [0.0, 2.0, 0.0]])
end_targets = ik_end_targets
ik_arms = []
for i in range(len(end_targets)):
    arm = fabrik_3d(arm, end_targets[i])
    ik_arms.append(arm)
ik_arms = np.array(ik_arms)
print("generating animations")
ani = utils.plot_nodes_trajs(ik_arms,"fabrik3d_fktargets.gif")
HTML(data=ani.to_jshtml())

## task 2 test case 1: 
- Run the test case below to test your solution
- You should be able to see the following gif after running the block successfully. (maybe different play speed)

 
![fabrik3d_oval.gif](fabrik3d_oval.gif)


In [ ]:
arm = np.array([[0.0, 0.0, 0.0], [0.0, 0.5, 0.0], [0.0, 1.0, 0.0], [0.0, 3.0, 0.0]])
end_targets = np.array(
    [[2 * np.cos(i / 100 * 2 * np.pi), np.sin(i / 100 * 2 * np.pi), -abs(i/100-1)+0.5] for i in range(200)]
)
ik_arms = []
for i in range(len(end_targets)):
    arm = fabrik_3d(arm, end_targets[i])
    ik_arms.append(arm)
ik_arms = np.array(ik_arms)
print("generating animations")
ani = utils.plot_nodes_trajs(ik_arms,"fabrik3d_oval.gif")
HTML(data=ani.to_jshtml())

# task 2 results generation
Run the following blocks that generate 2 gifs, "task2_1.gif" and "task2_2.gif"

In [ ]:
arm = np.array([[0.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 2.0, 0.0], [0.0, 3.0, 0.0]])
# generatea 3d heart as end targets
end_targets = np.array(
    [
        [
            1.6 * np.sin(i / 100 * 2 * np.pi) ** 3,
            1.3 * np.cos(i / 100 * 2 * np.pi) - 0.5 * np.cos(2 * i / 100 * 2 * np.pi)
            - 0.2 * np.cos(3 * i / 100 * 2 * np.pi) - 0.1 * np.cos(4 * i / 100 * 2 * np.pi),
            -abs(i / 100 - 1) + 0.5,
        ]
        for i in range(200)
    ]
)
ik_arms = []
for i in range(len(end_targets)):
    arm = fabrik_3d(arm, end_targets[i])
    ik_arms.append(arm)
ik_arms = np.array(ik_arms)
print("generating animations task2_1.gif")
ani = utils.plot_nodes_trajs(ik_arms,"task2_1.gif")
HTML(data=ani.to_jshtml())


In [ ]:
arm = np.array([[0.0, 0.0, 0.0], [0.0, 0.5, 0.0], [0.0, 1.2, 0.0], [0.0, 3.0, 0.0]])
# generatea 3d heart as end targets
end_targets = np.array(
    [
        [
            1.6 * np.sin(i / 100 * 2 * np.pi) ** 3,
            abs(i / 100 - 1) - 0.5,
            1.3 * np.cos(i / 100 * 2 * np.pi)
            - 0.5 * np.cos(2 * i / 100 * 2 * np.pi)
            - 0.2 * np.cos(3 * i / 100 * 2 * np.pi)
            - 0.1 * np.cos(4 * i / 100 * 2 * np.pi),
        ]
        for i in range(200)
    ]
)
ik_arms = []
for i in range(len(end_targets)):
    arm = fabrik_3d(arm, end_targets[i])
    ik_arms.append(arm)
ik_arms = np.array(ik_arms)
print("generating animations task2_2.gif")
ani = utils.plot_nodes_trajs(ik_arms, "task2_2.gif")
HTML(data=ani.to_jshtml())

# task 3: ccdik algorithm (40 points)
please implement your ccdik algorithm in the block below (within the implementation block).
1. your implementation should support ik in 3d space, including 3d arms and 3d target.(20 points)
2. your implementation should support bones with arbitrary length. (20 points)
- you can use utils.normalize to normalize a vector
- you can use utils.length to compute the length of a vector
- you can use utils.rotation_mat_3d to get the rotation matrix of a 3d rotation 
- you can use utils.angle_between3d to get the angle between 2 vectors.

> reference: https://alogicalmind.com/res/inverse_kinematics_ccd/paper.pdf

In [ ]:
def ccdik_3d(
        arm:np.ndarray, end_target:np.ndarray, max_iter:int = 1000,precision:float = 0.01
)->np.ndarray:
    """
    Implement the CCD algorithm for a 3D arm
    :param arm: np.ndarray of shape (n,3) representing the joints of the arm (the first joint is at the origin and static)
    :param end_target: np.ndarray of shape (3,) representing the end effector target
    :param max_iter: int, maximum number of iterations
    :param precision: float, tolerance for the distance between the end effector and the target
    :return: np.ndarray of shape (n,3) representing the new joints of the arm
    """
    num_joints = arm.shape[0]
    assert arm.shape == (num_joints, 3), f"Expected arm to be of shape {(num_joints, 3)} but got {arm.shape}"
    assert end_target.shape == (3,), f"Expected end_target to be of shape (3,) but got {end_target.shape}"

    ########## Implementation Block of CCDIK algorithm starts here ##########
    # your implementation, solution is 11 lines of code (not optimized for line count)
   
    ########## Implementation Block of CCDIK algorithm ends here ##########
    return arm

## task 3 test case 0: 
- Run the test case below to test your solution
- You should be able to see the following gif after running the block successfully. (maybe different play speed)

 
![ccdik3d_kftargets.gif](ccdik3d_fktargets.gif)

In [ ]:
arm = np.array([[0.0, 0.0, 0.0], [0.0, 0.5, 0.0], [0.0, 1.0, 0.0], [0.0, 3.0, 0.0]])
end_targets = ik_end_targets
ik_arms = []
for i in range(len(end_targets)):
    arm = ccdik_3d(arm, end_targets[i])
    ik_arms.append(arm)
ik_arms = np.array(ik_arms)
print("generating animations")
ani = utils.plot_nodes_trajs(ik_arms, "ccdik3d_fktargets.gif")
HTML(data=ani.to_jshtml())

## task 3 test case 1: 
- Run the test case below to test your solution
- You should be able to see the following gif after running the block successfully. (maybe different play speed)

 
![ccdik3d_oval.gif](ccdik3d_oval.gif)

In [ ]:
arm = np.array([[0.0, 0.0, 0.0], [0.0, 0.5, 0.0], [0.0, 1.0, 0.0], [0.0, 3.0, 0.0]])
end_targets = np.array(
    [
        [2 * np.cos(i / 100 * 2 * np.pi), np.sin(i / 100 * 2 * np.pi), -abs(i / 100 - 1) + 0.5]
        for i in range(200)
    ]
)
ik_arms = []
for i in range(len(end_targets)):
    arm = ccdik_3d(arm, end_targets[i])
    ik_arms.append(arm)
ik_arms = np.array(ik_arms)
print("generating animations")
ani = utils.plot_nodes_trajs(ik_arms, "ccdik3d_oval.gif")
HTML(data=ani.to_jshtml())

# task 3 results generation
Run the following blocks that generate 2 gifs, "task3_1.gif" and "task3_2.gif"

In [ ]:
arm = np.array([[0.0, 0.0, 0.0], [0.0, 1.0, 0.0],[0.0,2.0,0.0], [0.0, 3.0, 0.0]])
end_targets = np.array(
    [
        [ 1.5 * np.cos(i / 100 * 2 * np.pi),   2 * np.sin(1 + i / 100 * 2 * np.pi), -abs(i / 100 - 1) + 0.5]
        for i in range(200)
    ]
)

ik_arms = []
for i in range(len(end_targets)):
    arm = ccdik_3d(arm, end_targets[i])
    ik_arms.append(arm)
ik_arms = np.array(ik_arms)
print("generating animations")
ani = utils.plot_nodes_trajs(ik_arms, "task3_1.gif")
HTML(data=ani.to_jshtml())

In [ ]:
arm = np.array([[0.0, 0.0, 0.0], [0.0, 0.5, 0.0], [0.0, 1.0, 0.0], [0.0, 2.0, 0.0],[0.0,2.5,0.0]])
end_targets = np.array(
    [
        [1.5 * np.cos(i / 100 * 2 * np.pi), 2 * np.sin(1 + i / 100 * 2 * np.pi), -abs(i / 100 - 1) + 0.5]
        for i in range(200)
    ]
)

ik_arms = []
for i in range(len(end_targets)):
    arm = ccdik_3d(arm, end_targets[i])
    ik_arms.append(arm)
ik_arms = np.array(ik_arms)
print("generating animations")
ani = utils.plot_nodes_trajs(ik_arms, "task3_1.gif")
HTML(data=ani.to_jshtml())

# extra points:
Please come up a trajectory that the end effector of the arm can not reach any point on it. However, the max length of the arm > the distance from the root of the arm to any point on the trajectory

1. you should implement your own plot method that draws the end joint trajectory (green) and the target trajectory (red) with a gif

In [ ]:
arm = np.array([[0.0, 0.0, 0.0], [0.0, 3.0, 0.0], [0.0, 2.0, 0.0]])
########## Implementation Block of extra points starts here ##########
end_targets = None
########## Implementation Block of extra points ends here ##########
ik_arms = []
for i in range(len(end_targets)):
    arm = ccdik_3d(arm, end_target=end_targets[i])
    ik_arms.append(arm)
ik_arms = np.array(ik_arms)
print("generating animations")
########## Implementation Block of extra points starts here ##########
# generate your gif here with name "extra_points.gif"
########## Implementation Block of extra points ends here ##########